In [4]:
#Load google vectors through bin file

from gensim.models.word2vec import Word2Vec

model = Word2Vec.load_word2vec_format('Base/GoogleNews-vectors-negative300.bin', binary=True)


In [119]:
import numpy as np

with open('Base/food_words.txt', 'rb') as infile:
    food_words = infile.readlines()
    
with open('Base/sports_words.txt', 'rb') as infile:
    sport_words = infile.readlines()
    
with open('Base/weather_words.txt', 'rb') as infile:
    weather_words = infile.readlines()
    
uwords =[]

def getWordVecs(words):
    vecs = []
    for word in words:
        word = word.replace("\r\n", "")
        try:
            vecs.append(model[word])
            uwords.append(word)
        except KeyError:
            # When we have compound words, the result is the sum of the vectors of each word
            try:
                word = word.replace('-', ' ')
                nword = word.split(' ')
                vecs.append(sum(model[nword[0]], model[nword[1]]))
                #vecs.append(map(sum, zip(model[nword[0]], model[nword[1]])))
                uwords.append(word)
            except KeyError:
                print word
                continue
    #print len(vecs)
    #vecs = np.concatenate(vecs)
    #return vecs
    return np.array(vecs, dtype='float') #TSNE expects float type values

food_vecs = getWordVecs(food_words)
#sports_vecs = getWordVecs(sport_words)
#weather_vecs = getWordVecs(weather_words)

doughnut
gingerale
greenbean
soysauce
ugli fruit


In [ ]:
#Just for using less memory (Won't need load bin file)
#write whole vectors in a text file
with open('all_vectors_out.txt', 'w') as f:
    for row in food_vecs:
        f.write(str(row))
        f.write('\n')
    for row in sports_vecs:
        f.write(str(row))
        f.write('\n')
    for row in weather_vecs:
        f.write(str(row))
        f.write('\n')        
    f.close()
    
#Load text file
vectors = []
with open('all_vectors_out.txt', 'r') as a:
    all_vectors = [float(x) for x in a.readlines()]

In [161]:
from scipy.spatial.distance import *
import math

def WriteResult(Vector, Name_toExport):
    CRP_Words = CRP(Vector)
    with open(Name_toExport, 'w') as cluw:
        for w in CRP_Words:
            cluw.write(str(w))
            cluw.write('\n')
            cluw.write('\n')
        cluw.close()

def CRP(vector):    
    #Duas chaves tranforma em matriz 1X1
    clusterVec = [list(vector[0])]         # tracks sum of vectors in a cluster
    clusterIdx = [[0]]         # array of index arrays. e.g. [[1, 3, 5], [2, 4, 6]]
    ncluster = 0
    # probablity to create a new cluster vectors
    # is not strongly "similar" to any existing table
    pnew = 1.0/ (1+ ncluster) 
    N = len(vector)
    rands = np.random.rand(N)         # N rand variables sampled from U(0, 1)
 
    for i in range(N):
        maxSim = -1
        maxIdx = 0
        #List transforma em matriz normal
        v = list(vector[i])
        for j in range(ncluster):
            sim = 1- cosine(clusterVec[j], v)
            if math.isnan(sim):
                sim = 0
            if sim > maxSim:
                maxIdx = j
                maxSim = sim
        if maxSim < 2*pnew:                
            if rands[i] < pnew:
                clusterVec.append([])
                clusterIdx.append([])
                clusterVec[ncluster] = v
                clusterIdx[ncluster] = [i]
                ncluster += 1
                pnew = 1.0 / (1 + ncluster)
            else:
                clusterVec[maxIdx] = sum(clusterVec[maxIdx], v)
                #clusterVec[maxIdx] = map(sum, zip(clusterVec[maxIdx], v))
                clusterIdx[maxIdx].append(i)

        else:
            clusterVec[maxIdx] = sum(clusterVec[maxIdx], v)
            #clusterVec[maxIdx] = map(sum, zip(clusterVec[maxIdx], v))
            clusterIdx[maxIdx].append(i)
          
    ClusterWords = [[uwords[i] for i in c] for c in clusterIdx]
    return ClusterWords


In [160]:
WriteResult(food_vecs, 'Cluster_foodVectors')